## Function that returns bottleneck distance

In [ ]:
import ripser, persim
import numpy as np

def getdist(feature)
  '''
  Takes a 2D array with columns x,y
  '''
                  # Line has same width and number of X points as feature...          but with y values at zero
  line = np.array([np.linspace(min(feature[:,0]),max(feature[:,0]),len(feature[:,0])),np.zeros_like(feature[:,1])])
  feature_dgm = ripser.ripser(feature)['dgms'][0]
  line_dgm = ripser.ripser(line.T)['dgms'][0]
  return persim.bottleneck(feature_dgm, line_dgm)

## Function that loops over data and stores bottleneck distances in a higher order array

In [ ]:
def sliding_window():
    

## Read in data

In [ ]:
# Manually digitized natural features
spit = np.loadtxt('data/spit.txt')
cape = np.loadtxt('data/cape.txt')
inlet = np.loadtxt('data/inlet.txt')

# Video data
coasts = dict()
for fnum in range(1,6):
    fnBase = f"jgrf217-sup-000{fnum+1}-ms0{fnum}"
    readin = np.load(f"data/{fnBase}.npz")
    coasts[fnBase] = [readin[key] for key in readin.keys()]
    readin.close()
videoNum = 0
timestep = 0
# Returns 2D point cloud x,y
coasts[list(coasts.keys())[videoNum]][timestep]

## Run sliding windows

## Plot Heatmaps